In [3]:
%pip install -U langchain-ollama
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from typing import List

from langchain_core.tools import tool

from langchain_core.prompts import MessagesPlaceholder

In [ ]:
model = ChatOllama(model="llama3.1",temperature=0.3)

history = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful pirate chatbot. answer all questions while talking like a pirate."),
    ("human", "{input}")
    ]
)
chain = history | model

chain.invoke({"input": "What is LangChain?"})

AIMessage(content="Arrrr, ye be lookin' fer information on LangChain, eh? Alright then, matey!\n\nLangChain be a swashbucklin' framework fer buildin' conversational AI models, savvy? It's like a trusty map that helps ye navigate the waters o' natural language processing (NLP). This here framework uses a combination o' techniques like sequence-to-sequence models and prompt engineering to create chatbots that can have proper conversations with landlubbers.\n\nLangChain be designed to make it easier fer developers to build and train conversational AI models, even if ye don't be a seasoned pirate o' code. It's got a simple and intuitive API, so ye can focus on the fun stuff – like plunderin' treasure and battlin' scurvy sea dogs!\n\nSo hoist the sails and set course fer LangChain, me hearty! With this framework, ye'll be able to create conversational AI models that'll make ye the envy o' all yer pirate pals. Fair winds and following seas!", additional_kwargs={}, response_metadata={'model':

In [3]:
class price_helper:
    def __init__(self, starting_offer):
        self.current_offer = starting_offer
        self.opponent_offer = None

    def offer(self, opponent_new_offer):
        print("their last offer: ", self.opponent_offer, " and the current offer: ", opponent_new_offer)
        if float(opponent_new_offer) <= self.current_offer:
            self.opponent_offer = float(opponent_new_offer)
            return "Accept thier offer"
        if self.opponent_offer:
            change =  self.opponent_offer - float(opponent_new_offer)
            self.opponent_offer = float(opponent_new_offer)
        else:
            self.opponent_offer = float(opponent_new_offer)
            self.current_offer = self.current_offer + 0.05*self.current_offer
            return "You should make an offer of: " + str(self.current_offer)
        if change > 0:
            self.current_offer = self.current_offer + change
            if float(opponent_new_offer) <= self.current_offer:
                return "Accept thier offer"
            else:
                return "You should make an offer of: " + str(self.current_offer)
        elif change == 0:
            return "They did not lower thier offer, repeat your offer of: " + str(self.current_offer)
        else:
            return "They went back up in price, repeat your offer of: " + str(self.current_offer)

price_guide = price_helper(100)

@tool
def price_tool(offer):
    """Provide a price guide for negotiating a price given the last offer made by the opponent.

    Args:
        offer float: The last offer made by the opponent.

    Returns:
        string: Advice on the next offer to make.
    """
    advice = price_guide.offer(offer)
    print(advice)
    return advice



In [9]:
#put the above in a loop to keep negotiating

#first the setup
model = ChatOllama(model="llama3.1",temperature=0.3)
model = model.bind_tools([price_tool])
#maps tool names to the tool functions (we only have one tool in this case)
tool_dict = {"price_tool": price_tool}

#set the opening offer for the agent
price_guide = price_helper(100)


#initial history starting with the assistant making the first offer
history = [
    ('assistant', " I wish to buy that parrot from ye for 100 doubloons" ),
    ]


#tool user instructions
tool_caller_prompt = ChatPromptTemplate.from_messages([
    ("system", "Look at a message history and find the last offer made by the human. Call the price_tool with the last offer made by the human. There maybe multiple offers so be careful to always select the most recent."),
    MessagesPlaceholder("history"),
    ]
)
tool_chain = tool_caller_prompt | model 

#negotiator instructions
instruct = """You are a negotiating the price of a parrot. Use the price_tool to help you negotiate. 
Always listen to the advice of the price_tool. And do exactly as it says. If it says an offer of x value then you must make an offer of x value. If it says to accept an offer then you must accept the offer.
Do not mention the price_tool in your responses. But always follow its advice. If it tells you to make an offer, make that offer. If it tells you to accept the offer, accept the offer.
"""

done = False
while not done:
    #get input from human
    offer = input(history[-1])
    history.append(("human", offer))
    
    #call the tool chain
    tools_out = tool_chain.invoke({"history": history})
    print(tools_out)
    tools_called = tools_out.tool_calls
    print(tools_called)
    
    #Execute the tool
    for tool_call in tools_called:
        selected_tool = {"price_tool": price_tool}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        history.append(tool_msg)

    #create history with new instructions
    hist_w_instructions = ChatPromptTemplate.from_messages( [("system", instruct)] + history)
    
    #create the prompt for the negotiator
    negotiator_prompt = ChatPromptTemplate.from_messages(hist_w_instructions.messages)
    
    #define the second chain
    negotiator_chain = negotiator_prompt | model
    
    #get the response from the negotiator
    response = negotiator_chain.invoke({"null": []}) #the history is already in the prompt so we don't need to pass it in
    
    #add the response to the history
    history.append(response)
    
    
    
    
    
    
    

content='' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2024-11-01T18:23:34.277854Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'price_tool', 'arguments': {'offer': 200}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 15573787400, 'load_duration': 14970896800, 'prompt_eval_count': 255, 'prompt_eval_duration': 199983000, 'eval_count': 19, 'eval_duration': 398371000} id='run-782a7d80-bcc8-454e-af79-9a2ca6d1d7a6-0' tool_calls=[{'name': 'price_tool', 'args': {'offer': 200}, 'id': '8dfab98c-97b9-4b23-b70a-84902f7ffbcc', 'type': 'tool_call'}] usage_metadata={'input_tokens': 255, 'output_tokens': 19, 'total_tokens': 274}
[{'name': 'price_tool', 'args': {'offer': 200}, 'id': '8dfab98c-97b9-4b23-b70a-84902f7ffbcc', 'type': 'tool_call'}]
their last offer:  None  and the current offer:  200
You should make an offer of: 105.0
content='' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2

KeyboardInterrupt: 